<a href="https://colab.research.google.com/github/aleks-haksly/VIZRO/blob/main/demos/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sqlalchemy import text
from sqlalchemy import create_engine
import pandas as pd
from google.colab import userdata


engine = create_engine(userdata.get("supabase"), client_encoding='utf8', )

def select(sql):
    sql = text(sql)
    return pd.read_sql(sql, engine)

In [109]:
SQL = """
SELECT
    platform,
	query,
	count(*) AS cnt,
	sum(count(query)) over(PARTITION BY platform) AS platform_total,
	sum(count(query)) over(PARTITION BY query) AS query_count_total

FROM
	vizro.yandex_data yd
GROUP BY
	platform,
	query
"""

df = select(SQL)

In [110]:
min_query_cnt = 50
df = df.query("query_count_total >= @min_query_cnt")

In [116]:
df_pivoted = df.pivot(index='query', columns=["platform"], values=["cnt", "platform_total"],).reset_index()
df_pivoted.columns = ["_".join(a).rstrip('_') for a in df_pivoted.columns.to_flat_index()]

In [117]:
df_pivoted

,query,cnt_desktop,cnt_touch,platform_total_desktop,platform_total_touch
0,02 аниме,34.0,30.0,411566.0,792258.0
1,1 сентября,277.0,997.0,411566.0,792258.0
2,1 сентября 2021,19.0,71.0,411566.0,792258.0
3,1 сентября день знаний,69.0,137.0,411566.0,792258.0
4,1 сентября картинки,223.0,653.0,411566.0,792258.0
...,...,...,...,...,...
5083,ячмень на глазу лечение,12.0,56.0,411566.0,792258.0
5084,яэ мико,48.0,43.0,411566.0,792258.0
5085,ёжики,107.0,123.0,411566.0,792258.0
5086,سكس,6.0,77.0,411566.0,792258.0


In [118]:
df_pivoted.fillna(value={'cnt_touch':0, 'cnt_desktop':0, 'platform_total_desktop'	: df_pivoted['platform_total_desktop'].max(), 'platform_total_touch':df_pivoted['platform_total_touch'].max()}, inplace=True)

In [119]:
df_pivoted

,query,cnt_desktop,cnt_touch,platform_total_desktop,platform_total_touch
0,02 аниме,34.0,30.0,411566.0,792258.0
1,1 сентября,277.0,997.0,411566.0,792258.0
2,1 сентября 2021,19.0,71.0,411566.0,792258.0
3,1 сентября день знаний,69.0,137.0,411566.0,792258.0
4,1 сентября картинки,223.0,653.0,411566.0,792258.0
...,...,...,...,...,...
5083,ячмень на глазу лечение,12.0,56.0,411566.0,792258.0
5084,яэ мико,48.0,43.0,411566.0,792258.0
5085,ёжики,107.0,123.0,411566.0,792258.0
5086,سكس,6.0,77.0,411566.0,792258.0


In [38]:
from statsmodels.stats.proportion import proportions_chisquare

In [121]:
from statsmodels.stats.proportion import proportions_chisquare

def proportions_chi2(df: pd.DataFrame):

  _, pval, _ = proportions_chisquare(
  count=[df['cnt_desktop'], df['cnt_touch']],
  nobs=[df['platform_total_desktop'], df['platform_total_touch']])

  return pval

In [122]:
df_pivoted["pval"] = df_pivoted.apply(lambda x: proportions_chi2(x), axis=1)

In [123]:
df_pivoted

,query,cnt_desktop,cnt_touch,platform_total_desktop,platform_total_touch,pval
0,02 аниме,34.0,30.0,411566.0,792258.0,1.403780e-03
1,1 сентября,277.0,997.0,411566.0,792258.0,7.248046e-21
2,1 сентября 2021,19.0,71.0,411566.0,792258.0,8.908993e-03
3,1 сентября день знаний,69.0,137.0,411566.0,792258.0,8.338784e-01
4,1 сентября картинки,223.0,653.0,411566.0,792258.0,5.030819e-08
...,...,...,...,...,...,...
5083,ячмень на глазу лечение,12.0,56.0,411566.0,792258.0,4.031404e-03
5084,яэ мико,48.0,43.0,411566.0,792258.0,1.895721e-04
5085,ёжики,107.0,123.0,411566.0,792258.0,8.023864e-05
5086,سكس,6.0,77.0,411566.0,792258.0,2.241255e-07


In [124]:
df_pivoted['pct_desktop'] = df_pivoted['cnt_desktop'] / df_pivoted['platform_total_desktop']
df_pivoted['pct_touch'] = df_pivoted['cnt_touch'] / df_pivoted['platform_total_touch']

,query,cnt_desktop,cnt_touch,platform_total_desktop,platform_total_touch,pval,pct_desktop,pct_touch
3267,порно,1408.0,10076.0,411566.0,792258.0,0.000000e+00,0.003421,0.012718
3954,секс,947.0,8262.0,411566.0,792258.0,0.000000e+00,0.002301,0.010428
3785,с днём рождения женщине,503.0,4924.0,411566.0,792258.0,0.000000e+00,0.001222,0.006215
4736,хентай,821.0,4194.0,411566.0,792258.0,1.525804e-156,0.001995,0.005294
3768,с днём рождения,297.0,4122.0,411566.0,792258.0,0.000000e+00,0.000722,0.005203
...,...,...,...,...,...,...,...,...
324,texture,55.0,0.0,411566.0,792258.0,7.837933e-25,0.000134,0.000000
792,боруто: следующее поколение наруто мультсериал...,59.0,0.0,411566.0,792258.0,1.610809e-26,0.000143,0.000000
5070,яндекс.почта,178.0,0.0,411566.0,792258.0,1.649438e-76,0.000432,0.000000
145,facebook.com,79.0,0.0,411566.0,792258.0,6.074740e-35,0.000192,0.000000


In [90]:
!pip install vizro -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.0/857.0 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.8/500.8 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 14.8 MB/s eta 0:00:00


In [ ]:
import vizro.models as vm
import vizro.plotly.express as px
from vizro import Vizro
from vizro.tables import dash_ag_grid

df = df_pivoted.reset_index()

cellStyle = {
    "styleConditions": [
        {
            "condition": "params.value < 0.05",
            "style": {"backgroundColor": "#89ff22"},
        }
    ]
}

columnDefs = [
    {"field": "query"},
    {"field": "cnt_touch", "valueFormatter": {"function": "d3.format(',.0f')(params.value)"}},
    {"field": "pct_touch", "valueFormatter": {"function": "d3.format(',.2f')(params.value)"}},
    {"field": "cnt_desktop", "valueFormatter": {"function": "d3.format(',.0f')(params.value)"}},
    {"field": "pct_desktop", "valueFormatter": {"function": "d3.format(',.2f')(params.value)"}},
    {"field": "pval", "valueFormatter": {"function": "d3.format(',.3f')(params.value)",  "cellStyle": cellStyle,}},
]

page = vm.Page(
    title="Statisitical significencs in querries qty",
    components=[
        vm.AgGrid(
            title="Modified Dash AG Grid",
            figure=dash_ag_grid(
                data_frame=df,
                columnDefs=columnDefs,
                defaultColDef={"resizable": False, "filter": False, "editable": False},
            ),
        )
    ],
)

dashboard = vm.Dashboard(pages=[page])

Vizro().build(dashboard).run()